In [28]:
## Input Data Read
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator()
train_generator = datagen.flow_from_directory(
        'Dataset/train',
        batch_size=32,
        target_size=(224,224))
test_generator = datagen.flow_from_directory(
        'Dataset/test',
        batch_size=32,
        target_size=(224,224))

Found 280 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


In [32]:
## Load pretrained model
import h5py as h5py
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

# create the base pre-trained model
base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [33]:
import os
import numpy as np
import random as rn
import tensorflow as tf
from keras import backend as K
## To Obtain Reproducible result
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(16, activation='relu')(x)

# and a logistic layer -- let's say we have 2 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional ResNet50 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(train_generator,epochs=25,validation_data=test_generator)

Epoch 1/50
9/9 [==============================] - 177s 20s/step - loss: 1.4047 - acc: 0.5902
Epoch 2/50
9/9 [==============================] - 177s 20s/step - loss: 0.7446 - acc: 0.5954
Epoch 3/50
9/9 [==============================] - 180s 20s/step - loss: 0.6995 - acc: 0.5930
Epoch 4/50
9/9 [==============================] - 171s 19s/step - loss: 1.0928 - acc: 0.6710
Epoch 5/50
9/9 [==============================] - 172s 19s/step - loss: 0.7472 - acc: 0.6229
Epoch 6/50
9/9 [==============================] - 176s 20s/step - loss: 0.5193 - acc: 0.7093
Epoch 7/50
9/9 [==============================] - 181s 20s/step - loss: 0.5399 - acc: 0.7765
Epoch 8/50
9/9 [==============================] - 174s 19s/step - loss: 0.7086 - acc: 0.8054
Epoch 9/50
9/9 [==============================] - 172s 19s/step - loss: 0.6245 - acc: 0.7859
Epoch 10/50
9/9 [==============================] - 172s 19s/step - loss: 0.4434 - acc: 0.8412
Epoch 11/50
9/9 [==============================] - 173s 19s/step - lo

In [35]:
## Evaluate model
model.save('my_model.h5') 
model.predict_generator(test_generator)

array([[9.1217136e-01, 8.7828636e-02],
       [1.0000000e+00, 3.7171462e-18],
       [1.0000000e+00, 2.0504323e-25],
       [9.9985135e-01, 1.4866800e-04],
       [9.9755692e-01, 2.4430149e-03],
       [1.0000000e+00, 1.2180239e-18]], dtype=float32)